In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w4p2/sample.csv
/kaggle/input/2022-ai-w4p2/train.csv
/kaggle/input/2022-ai-w4p2/test.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/2022-ai-w4p2/train.csv')
test_data = pd.read_csv('/kaggle/input/2022-ai-w4p2/test.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder
cls = LabelEncoder()

train_data['label'] = cls.fit_transform(train_data['label'])
train_data

,N,P,K,temperature,humidity,ph,rainfall,label
0,22,36,16,30.581395,50.771481,8.184229,64.585596,13
1,18,27,41,22.365094,92.308824,7.175344,104.821633,19
2,61,68,50,35.214628,91.497251,6.793245,243.074507,17
3,26,72,22,28.767949,37.577921,4.674942,91.720849,18
4,2,140,197,22.697801,92.822234,5.534567,105.050823,0
...,...,...,...,...,...,...,...,...
1645,10,5,5,21.213070,91.353492,7.817846,112.983436,16
1646,108,94,47,27.359116,84.546250,6.387431,90.812505,1
1647,11,36,31,27.920633,51.779659,6.475449,100.258567,12
1648,11,124,204,13.429886,80.066340,6.361141,71.400430,7


In [4]:
train_data['label']

0       13
1       19
2       17
3       18
4        0
        ..
1645    16
1646     1
1647    12
1648     7
1649    10
Name: label, Length: 1650, dtype: int64

In [5]:
train_y = train_data['label']
train_x = train_data.drop('label', axis = 1)
train_x

,N,P,K,temperature,humidity,ph,rainfall
0,22,36,16,30.581395,50.771481,8.184229,64.585596
1,18,27,41,22.365094,92.308824,7.175344,104.821633
2,61,68,50,35.214628,91.497251,6.793245,243.074507
3,26,72,22,28.767949,37.577921,4.674942,91.720849
4,2,140,197,22.697801,92.822234,5.534567,105.050823
...,...,...,...,...,...,...,...
1645,10,5,5,21.213070,91.353492,7.817846,112.983436
1646,108,94,47,27.359116,84.546250,6.387431,90.812505
1647,11,36,31,27.920633,51.779659,6.475449,100.258567
1648,11,124,204,13.429886,80.066340,6.361141,71.400430


In [6]:
import torch
import torch.optim as optim

torch.manual_seed(1)
x_train = torch.FloatTensor(train_x.values)
x_test = torch.FloatTensor(test_data.values)
y_train = torch.LongTensor(train_y.values)


y_train

tensor([13, 19, 17,  ..., 12,  7, 10])

In [7]:
print(y_train.unsqueeze(1))

tensor([[13],
        [19],
        [17],
        ...,
        [12],
        [ 7],
        [10]])


In [8]:
print(y_train)
print(y_train.unsqueeze(1))
nb_class = 22
nb_data = len(y_train)

y_one_hot = torch.zeros(nb_data, nb_class)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

print(y_one_hot)

tensor([13, 19, 17,  ..., 12,  7, 10])
tensor([[13],
        [19],
        [17],
        ...,
        [12],
        [ 7],
        [10]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [9]:
import torch.nn.functional as F  # for softmax

nb_class = 22
nb_data = len(y_train)

W = torch.zeros((7, nb_class), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.0001)
nb_epochs = 29000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    
    # cost 표현번 1번 예시
    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    
    # cost 표현법 2번 예시
    # cross_entropy를 사용하면 scatter 함수를 이용한 one_hot_encoding을 안해도 됨
    #cost = F.cross_entropy((x_train.matmul(W) + b), y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/29000 Cost: 3.091043
Epoch 1000/29000 Cost: 0.837254
Epoch 2000/29000 Cost: 0.601120
Epoch 3000/29000 Cost: 0.492789
Epoch 4000/29000 Cost: 0.428235
Epoch 5000/29000 Cost: 0.384508
Epoch 6000/29000 Cost: 0.352506
Epoch 7000/29000 Cost: 0.327842
Epoch 8000/29000 Cost: 0.308112
Epoch 9000/29000 Cost: 0.291883
Epoch 10000/29000 Cost: 0.278241
Epoch 11000/29000 Cost: 0.266572
Epoch 12000/29000 Cost: 0.256448
Epoch 13000/29000 Cost: 0.247558
Epoch 14000/29000 Cost: 0.239675
Epoch 15000/29000 Cost: 0.232622
Epoch 16000/29000 Cost: 0.226267
Epoch 17000/29000 Cost: 0.220501
Epoch 18000/29000 Cost: 0.215240
Epoch 19000/29000 Cost: 0.210415
Epoch 20000/29000 Cost: 0.205970
Epoch 21000/29000 Cost: 0.201857
Epoch 22000/29000 Cost: 0.198038
Epoch 23000/29000 Cost: 0.194480
Epoch 24000/29000 Cost: 0.191154
Epoch 25000/29000 Cost: 0.188036
Epoch 26000/29000 Cost: 0.185106
Epoch 27000/29000 Cost: 0.182346
Epoch 28000/29000 Cost: 0.179739
Epoch 29000/29000 Cost: 0.177273


In [10]:
hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
hypothesis

tensor([[8.3855e-04, 1.2141e-04, 7.3750e-02,  ..., 4.7158e-04, 8.4097e-06,
         4.7212e-05],
        [3.3064e-04, 2.0144e-08, 2.2022e-07,  ..., 9.1949e-01, 2.7052e-07,
         2.4676e-06],
        [5.1154e-04, 6.2357e-06, 4.8805e-06,  ..., 1.5217e-06, 2.6587e-01,
         7.8421e-08],
        ...,
        [8.9316e-03, 1.3145e-05, 3.8897e-04,  ..., 3.8861e-03, 5.3688e-05,
         3.5753e-06],
        [5.7218e-02, 4.9775e-09, 1.4282e-25,  ..., 1.0341e-06, 2.0760e-28,
         9.5451e-17],
        [4.4975e-05, 2.3021e-04, 1.0120e-01,  ..., 4.8437e-08, 4.0261e-10,
         1.0297e-08]], grad_fn=<SoftmaxBackward0>)

In [11]:
predict = torch.argmax(hypothesis, dim=1)
print(hypothesis)
print(predict)
print(y_train)


# 정확도 계산 
correct_prediction = predict.float() == y_train
print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))


tensor([[8.3855e-04, 1.2141e-04, 7.3750e-02,  ..., 4.7158e-04, 8.4097e-06,
         4.7212e-05],
        [3.3064e-04, 2.0144e-08, 2.2022e-07,  ..., 9.1949e-01, 2.7052e-07,
         2.4676e-06],
        [5.1154e-04, 6.2357e-06, 4.8805e-06,  ..., 1.5217e-06, 2.6587e-01,
         7.8421e-08],
        ...,
        [8.9316e-03, 1.3145e-05, 3.8897e-04,  ..., 3.8861e-03, 5.3688e-05,
         3.5753e-06],
        [5.7218e-02, 4.9775e-09, 1.4282e-25,  ..., 1.0341e-06, 2.0760e-28,
         9.5451e-17],
        [4.4975e-05, 2.3021e-04, 1.0120e-01,  ..., 4.8437e-08, 4.0261e-10,
         1.0297e-08]], grad_fn=<SoftmaxBackward0>)
tensor([13, 19, 17,  ..., 12,  7, 10])
tensor([13, 19, 17,  ..., 12,  7, 10])
tensor([True, True, True,  ..., True, True, True])
The model has an accuracy of 95.76% for the training set.


In [12]:
hypothesis = F.softmax(x_test.matmul(W) + b, dim=1)
hypothesis

tensor([[3.1714e-10, 1.1600e-06, 1.1055e-10,  ..., 2.8246e-06, 5.4108e-13,
         1.9884e-02],
        [2.6731e-08, 2.1535e-05, 1.4616e-10,  ..., 2.1048e-04, 1.4793e-09,
         9.8861e-01],
        [2.1213e-03, 7.8475e-04, 2.4084e-03,  ..., 9.0727e-04, 2.3915e-07,
         1.8779e-04],
        ...,
        [4.6946e-03, 1.2458e-03, 1.1202e-10,  ..., 1.4128e-08, 8.7716e-11,
         1.8449e-08],
        [1.4043e-08, 4.1620e-05, 5.8025e-08,  ..., 1.4614e-09, 3.2859e-03,
         4.4911e-04],
        [1.1446e-04, 1.5700e-05, 2.3039e-05,  ..., 4.7781e-05, 3.1266e-01,
         1.6470e-05]], grad_fn=<SoftmaxBackward0>)

In [13]:
predict = torch.argmax(hypothesis, dim=1)
print(hypothesis)
print(predict)

tensor([[3.1714e-10, 1.1600e-06, 1.1055e-10,  ..., 2.8246e-06, 5.4108e-13,
         1.9884e-02],
        [2.6731e-08, 2.1535e-05, 1.4616e-10,  ..., 2.1048e-04, 1.4793e-09,
         9.8861e-01],
        [2.1213e-03, 7.8475e-04, 2.4084e-03,  ..., 9.0727e-04, 2.3915e-07,
         1.8779e-04],
        ...,
        [4.6946e-03, 1.2458e-03, 1.1202e-10,  ..., 1.4128e-08, 8.7716e-11,
         1.8449e-08],
        [1.4043e-08, 4.1620e-05, 5.8025e-08,  ..., 1.4614e-09, 3.2859e-03,
         4.4911e-04],
        [1.1446e-04, 1.5700e-05, 2.3039e-05,  ..., 4.7781e-05, 3.1266e-01,
         1.6470e-05]], grad_fn=<SoftmaxBackward0>)
tensor([15, 21, 17, 17,  0, 12,  0, 13, 14, 10,  2,  4, 19,  8,  4, 19,  0,  6,
        17, 15,  5, 17, 16, 17,  3,  8, 14, 16, 18, 20, 19, 10,  8, 10,  8,  2,
         8,  3,  3,  9, 17, 12,  2, 11, 14, 11, 18,  4, 15, 11,  2,  5,  7, 14,
         5,  9,  6,  0,  1, 10, 21,  4, 10, 16, 17,  2, 16, 20, 15, 18, 15,  4,
         8,  1,  2, 17,  1,  6, 21, 16,  5,  3,  8, 13, 

In [14]:
submit = pd.DataFrame()

submit['id'] = pd.Series(range(len(predict)))
submit['label'] = predict.detach().numpy()

submit

,id,label
0,0,15
1,1,21
2,2,17
3,3,17
4,4,0
...,...,...
545,545,5
546,546,17
547,547,3
548,548,5


In [15]:
submit['label'] = cls.inverse_transform(submit['label'].values)

In [16]:
submit

,id,label
0,0,muskmelon
1,1,watermelon
2,2,papaya
3,3,papaya
4,4,apple
...,...,...
545,545,coffee
546,546,papaya
547,547,chickpea
548,548,coffee


In [17]:
submit.to_csv('submit.csv', index = None)

In [18]:
# yo=